In [1]:
import asyncio
from pyppeteer import launch

In [29]:
execPath = 'C:\Program Files\Google\Chrome\Application\chrome.exe'
data_catalog ="Data/faq"
faq='https://akbf.ru/education-block/faq/general-issues'


In [2]:


browser = await launch(headless=False, executablePath=execPath) # Запускаем браузер
page = await browser.newPage() # Создаем новую вкладку
# Переходим на страницу для парсинга
await page.goto(faq)

# Исполняем JavaScript для получения заголовка страницы


In [10]:
import os
from urllib.parse import urlparse


if not os.path.exists(data_catalog):
    os.makedirs(data_catalog)

async def save_questions(page, title ):
    # title = await page.evaluate('() => document.title')
    url = await page.evaluate('() => window.location.href')
    parsed_url = urlparse(url)
    last_part = parsed_url.path.split('/')[-1]    # Находим элемент с классом, начинающимся на "content"
    element = await page.querySelector('[class^="content"]')
    if element:
        # Получаем текстовое содержимое элемента
        text_content = await page.evaluate('(element) => element.textContent', element)
        print('Содержимое элемента с классом, начинающимся на "content":', text_content[:100])
    else:
        print('Элемент с классом, начинающимся на "content", не найден')

    questions = await element.querySelectorAll('[class^="question"]')    
    lines = []
    lines.append(f"# {title}")
    for qe in questions:
        q = await qe.querySelector('[class^="q"]')
        a = await qe.querySelector('[class^="a"]')
        # Получаем текстовое содержимое каждой кнопки
        tq = await page.evaluate('(e) => e.textContent', q)
        ta = await page.evaluate('(e) => e.textContent', a)
        lines.append(f"## {tq}")
        lines.append(ta)

    # print(lines)


    with open(f"{data_catalog}/{last_part}.md","w",encoding="utf-8") as f:
        for line in lines:
                f.write(line + '\n')
            

In [12]:
fw = True
while fw:
    element = await page.querySelector('.chips___1AewC')
    if not element:
        break
    buttons = await element.querySelectorAll('button')
    fw = False
    f = False
    for button in buttons:
        # Получаем текстовое содержимое каждой кнопки
        if f:
            fw = True
            await button.click()
            # Ожидание обновления страницы
            await page.waitForNavigation()            
            break
        text_content = await page.evaluate('(button) => button.textContent', button)
        class_list = await page.evaluate('(button) => Array.from(button.classList)', button)
        if any(class_name.startswith('active') for class_name in class_list):
            title = await page.evaluate('(button) => button.textContent', button)
            await save_questions(page, title)
            f=True
        # print('Текст кнопки:', text_content)




Содержимое элемента с классом, начинающимся на "content": Какие инструменты торгуются на Фондовом рынке Московской бирже?Фондовый рынок Московской Биржи делит
Содержимое элемента с классом, начинающимся на "content": Как мне открыть счет?Вы можете открыть брокерский счёт, в том числе ИИС, дистанционно через сайт, на
Содержимое элемента с классом, начинающимся на "content": Кто такой инвестиционный советник?Статус инвестиционного советника, необходимый для оказания услуги 
Содержимое элемента с классом, начинающимся на "content": Что такое паевой инвестиционный фонд (ПИФ)?Паевой инвестиционный фонд (ПИФ, Фонд) — это форма коллек
Содержимое элемента с классом, начинающимся на "content": Для какого типа счета доступна услуга «Модельный портфель»?Услуга находится на стадии тестирования и
Содержимое элемента с классом, начинающимся на "content": Что такое доверительное управление (ДУ)?Доверительное управление – это деятельность по управлению це
Содержимое элемента с классом, начинающимся на

In [ ]:
url="https://akbf.ru/education-block/faq/comparison-iis-bs"
await page.goto(url)


In [18]:
url = await page.evaluate('() => window.location.href')
parsed_url = urlparse(url)
last_part = parsed_url.path.split('/')[-1]    # Находим элемент с классом, начинающимся на "content"
element = await page.querySelector('[class^="content"]')
if element:
    # Получаем текстовое содержимое элемента
    text_content = await page.evaluate('(element) => element.textContent', element)
    print('Содержимое элемента с классом, начинающимся на "content":', text_content[:100])
else:
    print('Элемент с классом, начинающимся на "content", не найден')

h = await element.querySelector('[class^="header"]')    
lines = []
lines.append(f"# {title}")
if h:
    pp = await h.querySelectorAll('[class^="typo"]')    
    f="## "
    for p in pp:
        th = await page.evaluate('(e) => e.textContent', p)
        lines.append(f+th)
        f=""

lines

Содержимое элемента с классом, начинающимся на "content": ИИС и брокерский счёт – в чем отличие?Отличие ИИС от брокерского счета, плюсы и минусы каждогоБрокер


['# ИИС III',
 '## ИИС и брокерский счёт – в чем отличие?',
 'Отличие ИИС от брокерского счета, плюсы и минусы каждого',
 'Брокерский счёт – это счет, открытый в организации, имеющей лицензию ЦБ РФ на осуществление брокерской деятельности, с целью торговли на бирже.',
 'ИИС – это частный случай брокерского счета, инвестирование через который дает больше возможностей получить инвестиционный налоговый вычет от государства']

In [19]:
t = await element.querySelector('[class^="table"]')   
hs = []
columns = []
if t:
    tcs = await t.querySelectorAll('[class^="table-column"]')
    for tc in tcs:
        th = await tc.querySelector('[class^="table-header"]')
        colt = ""
        items = []
        if th:
            colt = await page.evaluate('(e) => e.textContent', th)
        ti = await tc.querySelector('[class^="table-items"]') 
        if ti:
            tis = await ti.querySelectorAll('[class^="table-item"]')                   
            for i in tis:
                text = await page.evaluate('(e) => e.textContent', i)
                items.append(text)
        print(colt)
        print(items)
        hs.append(colt)
        columns.append(items)


Вид инвестиционного налогового вычета:
['В сумме денежных средств, внесенных в течение года на ИИС (налоговый вычет типа А)', 'В сумме положительного финансового результата, полученного по операциям, учитываемым на ИИС (налоговый вычет типа Б)', 'В размере положительного финансового результата, полученного в налоговом периоде от реализации (погашения) определенных ценных бумаг (долгосрочное владение)', 'Получение вычета, независимого от финансового результата', 'Освобождение от НДФЛ валютной переоценки еврооблигаций Минфина', 'Сальдирование результата с нескольких счетов', 'Срок существования счета', 'Частичный вывод средств', 'Ограничения на пополнение счета', 'Что можно внести на счет', 'Количество счетов', 'Что можно и нельзя купить на счет']
ИИС
['Возможен возврат НДФЛ до 52 000 ₽ в год, независимо от финансового результата по ИИС', 'Освобождение от уплаты НДФЛ полученного положительного финансового результата при владении ИИС не менее 3х лет', 'Не применимо', 'Предусмотрен налогов

In [22]:
data = []
col = columns[0]
for i in range(len(col)):
    row = []
    for j in range(len(columns)):
        row.append(columns[j][i])
    data.append(row)
table = {"header":hs, "data":data}
# table


In [23]:
import json
jt = json.dumps(table,ensure_ascii=False, indent=1)
with open(f"{data_catalog}/{last_part}.json","w",encoding="utf-8") as f:
    f.write(jt)

In [24]:

# print(lines)


with open(f"{data_catalog}/{last_part}.md","w",encoding="utf-8") as f:
    for line in lines:
            f.write(line + '\n')


In [28]:
files = os.listdir(data_catalog)
di = {}
for file in files:
    if file.endswith(".md"):
        with open(os.path.join(data_catalog, file), 'r', encoding='utf-8') as f:
            content = f.readline()
            di[file[:-3]]=content[2:-1]
di

{'adviser': 'Инвестиционный советник',
 'ak-bars-trade': 'АКБФ Трейд',
 'comparison-iis-bs': 'ИИС III',
 'general-issues': 'Общие вопросы',
 'individual-investment-account-3': 'ИИС III',
 'individual-investment-account': 'ИИС',
 'information-safety': 'Информационная безопасность',
 'lifehack-individual-investment-account': 'Лайфхаки ИИС',
 'margin-trading': 'Маржинальное кредитование',
 'model-portfolios': 'Модельный портфель',
 'mutual-funds': 'Паевые фонды',
 'quik': 'QUIK',
 'Tax': 'Налоги',
 'testinvest': 'Тестирование инвесторов',
 'trading-account': 'Брокерский счет',
 'trust-strategies': 'Доверительное управление'}